# Black Scholes Exercise 1: Naive implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [13]:
# Boilerplate for the example

import cProfile
import pstats
%load_ext line_profiler

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7_777_777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100_000
price, strike, t = gen_data(nopt)
call = [0.0 for i in range(nopt)]
put = [-1.0 for i in range(nopt)]
price=list(price)
strike=list(strike)
t=list(t)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# The Naive Black Scholes algorithm (looped)

In [14]:
from math import log, sqrt, exp, erf
import numpy as np
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = float( price [i] )
        S = strike [i]
        T = t [i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call [i] = P * d1 - Se * d2
        put [i] = call [i] - P + Se

# Timeit and CProfile Tests

What do you notice about the times?

In [15]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

268 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### cProfile is slower to run than timeit

In [16]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         600004 function calls in 0.394 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.024    0.000    0.036    0.000 <ipython-input-14-30645a7cb6d9>:3(<lambda>)
        1    0.264    0.264    0.394    0.394 <ipython-input-14-30645a7cb6d9>:5(black_scholes)
        1    0.000    0.000    0.394    0.394 <string>:1(<module>)
        1    0.000    0.000    0.394    0.394 {built-in method builtins.exec}
   200000    0.063    0.000    0.063    0.000 {built-in method math.erf}
   100000    0.012    0.000    0.012    0.000 {built-in method math.exp}
   100000    0.018    0.000    0.018    0.000 {built-in method math.log}
   100000    0.012    0.000    0.012    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# Line_Profiler tests

How many times does the function items get called (hits)?

%lprun -f function function(args)

In [17]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

Timer unit: 1e-06 s

Total time: 1.52035 s
File: <ipython-input-14-30645a7cb6d9>
Function: black_scholes at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
     6         1            2      2.0      0.0      mr = -rate
     7         1            2      2.0      0.0      sig_sig_two = vol * vol * 2
     8                                               
     9    100001        73589      0.7      4.8      for i in range(nopt):
    10    100000        96998      1.0      6.4          P = float( price [i] )
    11    100000        77356      0.8      5.1          S = strike [i]
    12    100000        74668      0.7      4.9          T = t [i]
    13                                                   
    14    100000        99329      1.0      6.5          a = log(P / S)
    15    100000        82602      0.8      5.4          b = T * mr
    16       